# Lesson 0: SQLite

The de facto language for managing relational databases is the Structured Query Language, or SQL ("sequel").

Many commerical and open-source relational data management systems (RDBMS) support SQL. The one we will consider in this class is the simplest, called [sqlite3](https://www.sqlite.org/). It stores the database in a simple file and can be run in a "standalone" mode from the command-line. However, we will, naturally, [invoke it from Python](https://docs.python.org/3/library/sqlite3.html). But all of the basic techniques apply to any commercial SQL backend.

With a little luck, you _might_ by the end of this class understand this [xkcd comic on SQL injection attacks](http://xkcd.com/327).

In [3]:
import sys
print("=== Python version info ===\n{}".format(sys.version))

=== Python version info ===
3.8.7 (default, Jan 25 2021, 11:14:52) 
[GCC 5.5.0 20171010]


## Getting started

In Python, you _connect_ to an `sqlite3` database by creating a _connection object_.

**Exercise 0** (ungraded). Run this code cell to get started.

In [4]:
import sqlite3 as db

# Connect to a database (or create one if it doesn't exist)
conn = db.connect('example.db')

The `sqlite` engine maintains a database as a file; in this example, the name of that file is `example.db`.

> **Important usage note!** If the named file does **not** yet exist, this code creates it. However, if the database has been created before, this same code will open it. This fact can be important when you are debugging. For example, if your code depends on the database not existing initially, then you may need to remove the file first. You're likely to encounter this situation in this notebook.

You issue commands to the database through an object called a _cursor_.

In [5]:
# Create a 'cursor' for executing commands
c = conn.cursor()

A cursor tracks the current state of the database, and you will mostly be using the cursor to issue commands that modify or query the database.

## Tables and Basic Queries

The central object of a relational database is a _table_. It's identical to what you called a "tibble" in the tidy data lab: observations as rows, variables as columns. In the relational database world, we sometimes refer to rows as _items_ or _records_ and columns as _attributes_. We'll use all of these terms interchangeably in this course.

Let's look at a concrete example. Suppose we wish to maintain a database of Georgia Tech students, whose attributes are their names and Georgia Tech-issued ID numbers. You might start by creating a table named `Students` to hold this data. You can create the table using the command, [`CREATE TABLE`](https://www.sqlite.org/lang_createtable.html).

> Note: If you try to create a table that already exists, it will **fail**. If you are trying to carry out these exercises from scratch, you may need to remove any existing `example.db` file or destroy any existing table; you can do the latter with the SQL command, `DROP TABLE IF EXISTS Students`.

In [6]:
# If this is not the first time you run this cell, 
# you need to delete the existed "Students" table first
c.execute("DROP TABLE IF EXISTS Students")

# create a table named "Students" with 2 columns: "gtid" and "name".
# the type for column "gtid" is integer and for "name" is text. 
c.execute("CREATE TABLE Students (gtid INTEGER, name TEXT)")

To populate the table with items, you can use the command, [`INSERT INTO`](https://www.sqlite.org/lang_insert.html).

In [7]:
c.execute("INSERT INTO Students VALUES (123, 'Vuduc')")
c.execute("INSERT INTO Students VALUES (456, 'Chau')")
c.execute("INSERT INTO Students VALUES (381, 'Bader')")
c.execute("INSERT INTO Students VALUES (991, 'Sokol')")

**Commitment issues.** The commands above modify the database. However, these are temporary modifications and aren't actually saved to the databases until you say so. (_Aside:_ Why would you want such behavior?) The way to do that is to issue a _commit_ operation from the _connection_ object.

> There are some subtleties related to when you actually need to commit, since the SQLite database engine does commit at certain points as discussed [here](https://stackoverflow.com/questions/13642956/commit-behavior-and-atomicity-in-python-sqlite3-module). However, it's probably simpler if you remember to encode commits when you intend for them to take effect.

In [8]:
conn.commit()

Another common operation is to perform a bunch of insertions into a table from a list of tuples. In this case, you can use `executemany()`.

In [9]:
# An important (and secure!) idiom
more_students = [(723, 'Rozga'),
                 (882, 'Zha'),
                 (401, 'Park'),
                 (377, 'Vetter'),
                 (904, 'Brown')]

# '?' question marks are placeholders for the two columns in Students table
c.executemany('INSERT INTO Students VALUES (?, ?)', more_students)
conn.commit()

Given a table, the most common operation is a _query_, which asks for some subset or transformation of the data. The simplest kind of query is called a [`SELECT`](https://www.sqlite.org/lang_select.html).

The following example selects all rows (items) from the `Students` table.

In [10]:
c.execute("SELECT * FROM Students")

results = c.fetchall()
print("Your results:", len(results), "\nThe entries of Students:\n", results)

Your results: 9 
The entries of Students:
 [(123, 'Vuduc'), (456, 'Chau'), (381, 'Bader'), (991, 'Sokol'), (723, 'Rozga'), (882, 'Zha'), (401, 'Park'), (377, 'Vetter'), (904, 'Brown')]


**Exercise 1** (2 points). Suppose we wish to maintain a second table, called `Takes`, which records classes that students have taken and the grades they earn.

In particular, each row of `Takes` stores a student by his/her GT ID, the course he/she took, and the grade he/she earned in terms of GPA (i.e. 4.0, 3.0, etc). More formally, suppose this table is defined as follows:

In [11]:
# Run this cell
c.execute('DROP TABLE IF EXISTS Takes')
c.execute('CREATE TABLE Takes (gtid INTEGER, course TEXT, grade REAL)')

Write a command to insert the following records into the `Takes` table.

* Vuduc: CSE 6040 - A (4.0), ISYE 6644 - B (3.0), MGMT 8803 - D (1.0)
* Sokol: CSE 6040 - A (4.0), ISYE 6740 - A (4.0)
* Chau: CSE 6040 - A (4.0), ISYE 6740 - C (2.0), MGMT 8803 - B (3.0)

(Note: See `students` table above to get the GT IDs for Vuduc, Sokol, and Chau. You don't have to write any code to retrieve their GT IDs. You can just type them in manually. However, it would be a good and extra practice for you if you can use some sql commands to retrieve their IDs.) 

In [12]:
# An important (and secure!) idiom
students = [(123, 'CSE 6040', 4.0),
            (123, 'ISYE 6644', 3.0),
            (123, 'MGMT 8803', 1.0),
            (991, 'CSE 6040', 4.0),
            (991, 'ISYE 6740', 4.0),
            (456, 'CSE 6040', 4.0),
            (456, 'ISYE 6740', 2.0),
            (456, 'MGMT 8803', 3.0),
            ]

# '?' question marks are placeholders for the two columns in Students table
c.executemany('INSERT INTO Takes VALUES (?, ?, ?)', students)
conn.commit()

# Displays the results of your code
c.execute('SELECT * FROM Takes')
results = c.fetchall()
print("Your results:", len(results), "\nThe entries of Takes:", results)

Your results: 8 
The entries of Takes: [(123, 'CSE 6040', 4.0), (123, 'ISYE 6644', 3.0), (123, 'MGMT 8803', 1.0), (991, 'CSE 6040', 4.0), (991, 'ISYE 6740', 4.0), (456, 'CSE 6040', 4.0), (456, 'ISYE 6740', 2.0), (456, 'MGMT 8803', 3.0)]


In [13]:
# Test cell: `insert_many__test`

# Close the database and reopen it
conn.close()
conn = db.connect('example.db')
c = conn.cursor()
c.execute('SELECT * FROM Takes')
results = c.fetchall()

if len(results) == 0:
    print("*** No matching records. Did you remember to commit the results? ***")
assert len(results) == 8, "The `Takes` table has {} when it should have {}.".format(len(results), 8)

assert (123, 'CSE 6040', 4.0) in results
assert (123, 'ISYE 6644', 3.0) in results
assert (123, 'MGMT 8803', 1.0) in results
assert (991, 'CSE 6040', 4.0) in results
assert (991, 'ISYE 6740', 4.0) in results
assert (456, 'CSE 6040', 4.0) in results
assert (456, 'ISYE 6740', 2.0) in results
assert (456, 'MGMT 8803', 3.0) in results

print("\n(Passed.)")


(Passed.)


# Lesson 1: Join queries

The main type of query that combines information from multiple tables is the _join query_. Recall from our discussion of tibbles these four types:

- `INNER JOIN(A, B)`: Keep rows of `A` and `B` only where `A` and `B` match
- `OUTER JOIN(A, B)`: Keep all rows of `A` and `B`, but merge matching rows and fill in missing values with some default (`NaN` in Pandas, `NULL` in SQL)
- `LEFT JOIN(A, B)`: Keep all rows of `A` but only merge matches from `B`.
- `RIGHT JOIN(A, B)`: Keep all rows of `B` but only merge matches from `A`.

If you are a visual person, see [this page](https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins) for illustrations of the different join types.

In SQL, you can use the `WHERE` clause of a `SELECT` statement to specify how to match rows from the tables being joined. For example, recall that the `Takes` table stores classes taken by each student. However, these classes are recorded by a student's GT ID. Suppose we want a report where we want each student's name rather than his/her ID. We can get the matching name from the `Students` table. Here is a query to accomplish this matching:

In [14]:
# See all (name, course, grade) tuples
query = '''
        SELECT Students.name, Takes.course, Takes.grade
        FROM Students, Takes
        WHERE Students.gtid = Takes.gtid
'''

for match in c.execute(query): # Note this alternative idiom for iterating over query results
    print(match)

('Vuduc', 'CSE 6040', 4.0)
('Vuduc', 'ISYE 6644', 3.0)
('Vuduc', 'MGMT 8803', 1.0)
('Chau', 'CSE 6040', 4.0)
('Chau', 'ISYE 6740', 2.0)
('Chau', 'MGMT 8803', 3.0)
('Sokol', 'CSE 6040', 4.0)
('Sokol', 'ISYE 6740', 4.0)


**Exercise 2** (2 points). Define a query to select the names and grades of students where the course is _CSE 6040_. The code below will execute your query and store the results in a list `results1` of tuples, where each tuple is a `(name, grade)` pair; thus, you should structure your query to match this format.

In [15]:
# Define `query` with your query:
###
### YOUR CODE HERE
###
query = ''' 
        select Students.name, Takes.grade
        from Students, Takes
        where Students.gtid = Takes.gtid and Takes.course = 'CSE 6040'

    '''


c.execute(query)
results1 = c.fetchall()
results1

[('Vuduc', 4.0), ('Sokol', 4.0), ('Chau', 4.0)]

In [16]:
# Test cell: `join1__test`

print ("Your results:", results1)

assert type(results1) is list
assert len(results1) == 3, "Your query produced {} results instead of {}.".format(len(results1), 3)

assert set(results1) == {('Vuduc', 4.0), ('Sokol', 4.0), ('Chau', 4.0)}

print("\n(Passed.)")

Your results: [('Vuduc', 4.0), ('Sokol', 4.0), ('Chau', 4.0)]

(Passed.)


For contrast, let's do a quick exercise that executes a [left join](http://www.sqlitetutorial.net/sqlite-left-join/).

**Exercise 3** (2 points). Execute a `LEFT JOIN` that uses `Students` as the left table, `Takes` as the right table, and selects a student's name and course grade. Write your query as a string variable named `query`, which the subsequent code will execute.

In [17]:
# Define `query` string here:
###
### YOUR CODE HERE
###

query = '''
        SELECT Students.name, Takes.grade
        FROM Students LEFT JOIN Takes 
        ON Students.gtid = Takes.gtid
        '''

# Executes your `query` string:
c.execute(query)
matches = c.fetchall()
for i, match in enumerate(matches):
    print(i, "->", match)

0 -> ('Vuduc', 1.0)
1 -> ('Vuduc', 3.0)
2 -> ('Vuduc', 4.0)
3 -> ('Chau', 2.0)
4 -> ('Chau', 3.0)
5 -> ('Chau', 4.0)
6 -> ('Bader', None)
7 -> ('Sokol', 4.0)
8 -> ('Sokol', 4.0)
9 -> ('Rozga', None)
10 -> ('Zha', None)
11 -> ('Park', None)
12 -> ('Vetter', None)
13 -> ('Brown', None)


In [18]:
# Test cell: `left_join_test`

assert set(matches) == {('Vuduc', 4.0), ('Chau', 2.0), ('Park', None), ('Vuduc', 1.0), ('Chau', 3.0), ('Zha', None), ('Brown', None), ('Vetter', None), ('Vuduc', 3.0), ('Bader', None), ('Rozga', None), ('Chau', 4.0), ('Sokol', 4.0)}
print("\n(Passed!)")


(Passed!)


## Aggregations

Another common style of query is an [_aggregation_](https://www.sqlite.org/lang_aggfunc.html), which is a summary of information across multiple records, rather than the raw records themselves.

For instance, suppose we want to compute the average GPA for each unique GT ID from the `Takes` table. Here is a query that does it using `AVG` aggregator:

In [19]:
query = '''
        SELECT gtid, AVG(grade)
        FROM Takes 
        GROUP BY gtid
'''

for match in c.execute(query):
    print(match)

(123, 2.6666666666666665)
(456, 3.0)
(991, 4.0)


Some other useful SQL aggregators include `MIN`, `MAX`, `SUM`, and `COUNT`.

## Cleanup

As one final bit of information, it's good practice to shutdown the cursor and connection, the same way you close files.

In [20]:
c.close()
conn.close()

**What next?** It's now a good time to look at a different tutorial which reviews this material and introduces some additional topics: [A thorough guide to SQLite database operations in Python](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html).

In [21]:
from IPython.display import display
import pandas as pd
import sqlite3 as db

# Connect to o database (or creote one if it doesn't exist)
conn = db.connect('example.db')

df_students = pd.read_sql_query ('select * from Students',conn)
df_takes = pd.read_sql_query ('select * from Takes',conn)

In [22]:
display(df_students)
display(df_takes)

,gtid,name
0,123,Vuduc
1,456,Chau
2,381,Bader
3,991,Sokol
4,723,Rozga
5,882,Zha
6,401,Park
7,377,Vetter
8,904,Brown


,gtid,course,grade
0,123,CSE 6040,4.0
1,123,ISYE 6644,3.0
2,123,MGMT 8803,1.0
3,991,CSE 6040,4.0
4,991,ISYE 6740,4.0
5,456,CSE 6040,4.0
6,456,ISYE 6740,2.0
7,456,MGMT 8803,3.0


In [23]:
# first n records
print('#### SQL ####')
db = pd.read_sql_query('select * from Students limit 5', conn)
display(db)

print('#### Pandas DataFrame ####')
df = df_students.head(5)
display(df)


#### SQL ####


,gtid,name
0,123,Vuduc
1,456,Chau
2,381,Bader
3,991,Sokol
4,723,Rozga


#### Pandas DataFrame ####


,gtid,name
0,123,Vuduc
1,456,Chau
2,381,Bader
3,991,Sokol
4,723,Rozga


In [24]:
# filter records
print('#### SQL - will rename the index ####')
db = pd.read_sql_query("select * from Takes WHERE course = 'CSE 6040'", conn)
display(db)

print('#### Pandas DataFrame - will keep original index####')
df = df_takes[df_takes['course']=='CSE 6040']
display(df)

#### SQL - will rename the index ####


,gtid,course,grade
0,123,CSE 6040,4.0
1,991,CSE 6040,4.0
2,456,CSE 6040,4.0


#### Pandas DataFrame - will keep original index####


,gtid,course,grade
0,123,CSE 6040,4.0
3,991,CSE 6040,4.0
5,456,CSE 6040,4.0


In [25]:
# filter records IN
print('#### SQL - will rename the index ####')
db = pd.read_sql_query("select * from Takes WHERE course IN ('CSE 6040', 'MGMT 8803')", conn)
display(db)

print('#### Pandas DataFrame - will keep original index ####')
df = df_takes[df_takes['course'].isin(['CSE 6040'])]
display(df)

#### SQL - will rename the index ####


,gtid,course,grade
0,123,CSE 6040,4.0
1,123,MGMT 8803,1.0
2,991,CSE 6040,4.0
3,456,CSE 6040,4.0
4,456,MGMT 8803,3.0


#### Pandas DataFrame - will keep original index ####


,gtid,course,grade
0,123,CSE 6040,4.0
3,991,CSE 6040,4.0
5,456,CSE 6040,4.0


In [26]:
# sort values
print('#### SQL - will rename the index ####')
# desc = big -> small
db = pd.read_sql_query("select * from Takes ORDER BY grade desc", conn)
display(db)

print('#### Pandas DataFrame - will keep original index ####')
df = df_takes.sort_values('grade', ascending = False)
display(df)

#### SQL - will rename the index ####


,gtid,course,grade
0,123,CSE 6040,4.0
1,991,CSE 6040,4.0
2,991,ISYE 6740,4.0
3,456,CSE 6040,4.0
4,123,ISYE 6644,3.0
5,456,MGMT 8803,3.0
6,456,ISYE 6740,2.0
7,123,MGMT 8803,1.0


#### Pandas DataFrame - will keep original index ####


,gtid,course,grade
0,123,CSE 6040,4.0
3,991,CSE 6040,4.0
4,991,ISYE 6740,4.0
5,456,CSE 6040,4.0
1,123,ISYE 6644,3.0
7,456,MGMT 8803,3.0
6,456,ISYE 6740,2.0
2,123,MGMT 8803,1.0


In [27]:
# unique values
print('#### SQL - will rename the index ####')
db = pd.read_sql_query("select DISTINCT course from Takes", conn)
display(db)

print('#### Pandas DataFrame - will keep original index ####')
df = df_takes['course'].unique()
display(df)

#### SQL - will rename the index ####


,course
0,CSE 6040
1,ISYE 6644
2,MGMT 8803
3,ISYE 6740


#### Pandas DataFrame - will keep original index ####


array(['CSE 6040', 'ISYE 6644', 'MGMT 8803', 'ISYE 6740'], dtype=object)

In [40]:
# value counts

print('#### SQL - will rename the index ####')
sql = '''
        select course, count(*) as course_count 
        from Takes 
        Group by course 
        order by course_count desc
     '''


db = pd.read_sql_query(sql, conn)
display(db)

print('#### Pandas DataFrame - will keep original index ####')
df = df_takes['course'].value_counts()
display(df)

#### SQL - will rename the index ####


,course,course_count
0,CSE 6040,3
1,MGMT 8803,2
2,ISYE 6740,2
3,ISYE 6644,1


#### Pandas DataFrame - will keep original index ####


CSE 6040     3
MGMT 8803    2
ISYE 6740    2
ISYE 6644    1
Name: course, dtype: int64

In [30]:
# group by sum

print('#### SQL - will rename the index ####')
sql = '''
       select course, sum(grade) as grade_sum\
       from Takes \
       group by course
    '''
db = pd.read_sql_query(sql, conn)
display(db)

print('#### Pandas DataFrame - will keep original index ####')
df = df_takes.groupby('course')['grade'].sum()
display(df)

# group by max

# print('#### SQL - will rename the index ####')
# sql = '''select course, MAX(grade) as grade_sum
#        from Takes 
#        group by course'''
# db = pd.read_sql_query(sql, conn)
# display(db)

# print('#### Pandas DataFrame - will keep original index ####')
# df = df_takes.groupby('course')['grade'].max()
# display(df)

#### SQL - will rename the index ####


,course,grade_sum
0,CSE 6040,12.0
1,ISYE 6644,3.0
2,ISYE 6740,6.0
3,MGMT 8803,4.0


#### Pandas DataFrame - will keep original index ####


course
CSE 6040     12.0
ISYE 6644     3.0
ISYE 6740     6.0
MGMT 8803     4.0
Name: grade, dtype: float64

In [41]:
# join tables
# the main one is Takes table
print('#### SQL - will rename the index ####')
sql = '''select t1.* , t2.* 
        from Students as t1 
          LEFT JOIN Takes as t2 
            ON t1.gtid = t2.gtid '''
db = pd.read_sql_query(sql, conn)
display(db)

print('#### Pandas DataFrame - will keep original index ####')
df = df_students.merge(df_takes, how = 'left', on = 'gtid')
display(df)

#### SQL - will rename the index ####


,gtid,name,gtid,course,grade
0,123,Vuduc,123.0,CSE 6040,4.0
1,123,Vuduc,123.0,ISYE 6644,3.0
2,123,Vuduc,123.0,MGMT 8803,1.0
3,456,Chau,456.0,CSE 6040,4.0
4,456,Chau,456.0,ISYE 6740,2.0
5,456,Chau,456.0,MGMT 8803,3.0
6,381,Bader,NaN,None,NaN
7,991,Sokol,991.0,CSE 6040,4.0
8,991,Sokol,991.0,ISYE 6740,4.0
9,723,Rozga,NaN,None,NaN


#### Pandas DataFrame - will keep original index ####


,gtid,name,course,grade
0,123,Vuduc,CSE 6040,4.0
1,123,Vuduc,ISYE 6644,3.0
2,123,Vuduc,MGMT 8803,1.0
3,456,Chau,CSE 6040,4.0
4,456,Chau,ISYE 6740,2.0
5,456,Chau,MGMT 8803,3.0
6,381,Bader,NaN,NaN
7,991,Sokol,CSE 6040,4.0
8,991,Sokol,ISYE 6740,4.0
9,723,Rozga,NaN,NaN


In [38]:
# join tables and filter the result
print('#### SQL - will rename the index ####')
sql = '''
    select t1.* , t2.* 
        from Students as t1 
          LEFT JOIN Takes as t2 
            ON t1.gtid = t2.gtid 
        WHERE t2.gtid is NOT NULL 
    '''

db = pd.read_sql_query(sql, conn)
display(db)

print('#### Pandas DataFrame - will keep original index ####')
df = df_students.merge(df_takes, how = 'left', on = 'gtid')
df = df[df['course'].notnull()]
# reset index
df.reset_index(inplace = True)
display(df)

#### SQL - will rename the index ####


,gtid,name,gtid,course,grade
0,123,Vuduc,123,CSE 6040,4.0
1,123,Vuduc,123,ISYE 6644,3.0
2,123,Vuduc,123,MGMT 8803,1.0
3,991,Sokol,991,CSE 6040,4.0
4,991,Sokol,991,ISYE 6740,4.0
5,456,Chau,456,CSE 6040,4.0
6,456,Chau,456,ISYE 6740,2.0
7,456,Chau,456,MGMT 8803,3.0


#### Pandas DataFrame - will keep original index ####


,index,gtid,name,course,grade
0,0,123,Vuduc,CSE 6040,4.0
1,1,123,Vuduc,ISYE 6644,3.0
2,2,123,Vuduc,MGMT 8803,1.0
3,3,456,Chau,CSE 6040,4.0
4,4,456,Chau,ISYE 6740,2.0
5,5,456,Chau,MGMT 8803,3.0
6,7,991,Sokol,CSE 6040,4.0
7,8,991,Sokol,ISYE 6740,4.0


In [45]:
# nested query
print('#### SQL - regular query ####')
sql = ''' select t1.name, t2.*
          from Students t1
            LEFT join Takes t2
            ON t1.gtid = t2.gtid
        '''
db = pd.read_sql_query(sql, conn)
display(db)

# create a new table and use the nested data to manipulate the new table 
print('#### SQL - nested query ####')
sql = ''' select *
          from (
              select t1.name, t2.*
              from Students t1
                LEFT JOIN Takes t2
                ON t1.gtid = t2.gtid
                )
          where gtid is not null
        '''
db = pd.read_sql_query(sql, conn)
display(db)

#### SQL - regular query ####


,name,gtid,course,grade
0,Vuduc,123,CSE 6040,4.0
1,Vuduc,123,ISYE 6644,3.0
2,Vuduc,123,MGMT 8803,1.0
3,Sokol,991,CSE 6040,4.0
4,Sokol,991,ISYE 6740,4.0
5,Chau,456,CSE 6040,4.0
6,Chau,456,ISYE 6740,2.0
7,Chau,456,MGMT 8803,3.0


#### SQL - nested query ####


,name,gtid,course,grade
0,Vuduc,123,CSE 6040,4.0
1,Vuduc,123,ISYE 6644,3.0
2,Vuduc,123,MGMT 8803,1.0
3,Chau,456,CSE 6040,4.0
4,Chau,456,ISYE 6740,2.0
5,Chau,456,MGMT 8803,3.0
6,Sokol,991,CSE 6040,4.0
7,Sokol,991,ISYE 6740,4.0


In [ ]:
# LETTERS & TIME 
sql = '''
        select LOWER(ComplainType), CreatedDate, UPPER(City)
        from data
        where CreatedDate >= "2015-09-15 00:00:00.0"
          and CreatedDate < "2015-09-16 00:00:00.0"
        orderby CreatedDate
    '''

sql = '''
        select CreatedDate, STRFTIME('%H', CreatedDate) as Hour,
                LOWER(ComplainType)
        from data
        limit 5
    '''
